In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    model_name="llama-3.1-70b-versatile"
)

response = llm.invoke("the first person to land on moon was...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [2]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-38999")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Apply for Software Engineer III

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologySoftware Engineer IIIBeaverton, OregonBecome a Part of the NIKE, Inc. TeamNIKE, Inc. does more 

In [3]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
      """
      ### SCRAPED TEXT FROM WEBSITE:
      {page_data}
      ### INSTRUCTION:
      The scraped text is from the career's page of a website.
      Your job is to extract the job posting and return the in JSON format containing the 
      following keys : 'role','experience','skills' and 'description'.
      Only return the valid JSON.
      ### VALID JSON (NO PREAMBLE):
      """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

```
{
  "role": "Software Engineer III",
  "experience": "4+ years of professional experience in application engineering",
  "skills": [
    "JavaScript (Node.js, React.js, Vue.js, etc.)",
    "Microservice architecture",
    "Domain driven design",
    "Restful APIs",
    "NoSQL data stores (DynamoDB, MongoDB)",
    "Cloud services (ECS, S3, Lambdas)",
    "Version control systems (Git)",
    "Issue tracking tools (Jira)",
    "CI/CD tools (Jenkins, Terraform)",
    "AWS CloudFront",
    "AWS S3",
    "AWS Route 53",
    "AWS Certificate Manager",
    "AWS Lambda",
    "GitHub Co-pilot"
  ],
  "description": "The Digital Employee Experience Software Engineer III will be responsible for design, support, development, and testing of internal employee-facing websites and web applications."
}
```


In [4]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Software Engineer III',
 'experience': '4+ years of professional experience in application engineering',
 'skills': ['JavaScript (Node.js, React.js, Vue.js, etc.)',
  'Microservice architecture',
  'Domain driven design',
  'Restful APIs',
  'NoSQL data stores (DynamoDB, MongoDB)',
  'Cloud services (ECS, S3, Lambdas)',
  'Version control systems (Git)',
  'Issue tracking tools (Jira)',
  'CI/CD tools (Jenkins, Terraform)',
  'AWS CloudFront',
  'AWS S3',
  'AWS Route 53',
  'AWS Certificate Manager',
  'AWS Lambda',
  'GitHub Co-pilot'],
 'description': 'The Digital Employee Experience Software Engineer III will be responsible for design, support, development, and testing of internal employee-facing websites and web applications.'}

In [5]:
import pandas as pd
df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [6]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [7]:
links = collection.query(query_texts=['Experience in python','Experience in React','experience in react native'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}],
 [{'links': 'https://example.com/react-native-portfolio'},
  {'links': 'https://example.com/react-portfolio'}]]

In [10]:
job

{'role': 'Software Engineer III',
 'experience': '4+ years of professional experience in application engineering',
 'skills': ['JavaScript (Node.js, React.js, Vue.js, etc.)',
  'Microservice architecture',
  'Domain driven design',
  'Restful APIs',
  'NoSQL data stores (DynamoDB, MongoDB)',
  'Cloud services (ECS, S3, Lambdas)',
  'Version control systems (Git)',
  'Issue tracking tools (Jira)',
  'CI/CD tools (Jenkins, Terraform)',
  'AWS CloudFront',
  'AWS S3',
  'AWS Route 53',
  'AWS Certificate Manager',
  'AWS Lambda',
  'GitHub Co-pilot'],
 'description': 'The Digital Employee Experience Software Engineer III will be responsible for design, support, development, and testing of internal employee-facing websites and web applications.'}

In [9]:
job = json_res
job['skills']

['JavaScript (Node.js, React.js, Vue.js, etc.)',
 'Microservice architecture',
 'Domain driven design',
 'Restful APIs',
 'NoSQL data stores (DynamoDB, MongoDB)',
 'Cloud services (ECS, S3, Lambdas)',
 'Version control systems (Git)',
 'Issue tracking tools (Jira)',
 'CI/CD tools (Jenkins, Terraform)',
 'AWS CloudFront',
 'AWS S3',
 'AWS Route 53',
 'AWS Certificate Manager',
 'AWS Lambda',
 'GitHub Co-pilot']

In [11]:
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}
    
    ### INSTRUCTION:
    You are Hardik Tater a final year BTech CSE Student You are a motivated student or fresh graduate eager to apply for the mentioned position. 
    In your email, briefly introduce yourself, highlighting your skills and how they align with the job. 
    Mention your academic achievements and relevant projects you've worked on that demonstrate your fit for the role.
    Also add the most relevant ones from the following links to showcase My portfolio: {link_list}
    Remember you are Hardik. 
    Sign off with an eagerness to discuss the opportunity further.
    ### EMAIL (NO PREAMBLE):
    
    """
)

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Application for Software Engineer III Position

Dear Hiring Manager,

I am Hardik Tater, a final year BTech CSE student with a strong passion for software development and a keen interest in the role of Software Engineer III at your esteemed organization. With a solid foundation in computer science and a drive to learn and grow, I am confident that my skills and experience make me an ideal fit for this position.

Throughout my academic journey, I have developed a strong proficiency in programming languages such as JavaScript, Python, and Java. I have also gained hands-on experience with various technologies, including Node.js, React.js, and Vue.js, which align with the job requirements. My understanding of microservice architecture, domain-driven design, and Restful APIs has been further solidified through my involvement in various projects and academic assignments.

In my previous projects, I have worked on designing and developing web applications using React.js and Node.js. 